In [1]:
import os
import re
import pickle as pkl
import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

import scorpyo as sp

from null_importance import get_null_importance


pd.set_option('max_rows', 500, 'max_columns', 200)

In [2]:
path_project = r'/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification'

# path dir
path_row_data = os.path.join(path_project, 'row_data')
path_new_data = os.path.join(path_project, 'new_data')
path_results  = os.path.join(path_project, 'results')
path_results_jupyter  = os.path.join(path_results, 'jupyter')

# path row_data
path_train = os.path.join(path_row_data, 'train.csv')
path_test  = os.path.join(path_row_data, 'evaluation_public.csv')
path_sample_submission = os.path.join(path_row_data, 'submit_example.csv')


path_new_train = os.path.join(path_new_data, 'train_lightgbm_20221014.csv')
path_new_test  = os.path.join(path_new_data, 'test_lightgbm_20221014.csv')

## results
path_output_report = os.path.join(path_results, '01_原始数据探察_20221014.xlsx')

y_label = "is_risk"

In [3]:
df_row_train = sp.read_data(path_train)
df_row_val  = sp.read_data(path_test)

df_row_train['url_sit'] = df_row_train['url'].map(lambda x: x.split('/')[0])
df_row_train['url_page'] = df_row_train['url'].map(lambda x: x.split('/')[1])

df_row_val['url_sit'] = df_row_val['url'].map(lambda x: x.split('/')[0])
df_row_val['url_page'] = df_row_val['url'].map(lambda x: x.split('/')[1])



df_train_info = df_row_train.copy().sort_values(by='op_datetime')

In [8]:
df_train_info.head()

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,op_datetime,ip_type,http_status_code,op_city,log_system_transform,url,op_month,is_risk,url_sit,url_page
44477,44477,xiongkai3397,rd,6H1iPLgBB,GCgxrFb69up7,chrome_93,chrome,win,win10,2022-01-07 02:44:29,内网,200,深圳,nHrKgKdJ1Mzt,xxx.com/github,2022-01,1,xxx.com,github
45489,45489,zhengguiying7117,rd,0mjaEf4SB,8ftsXFm5I1Ej,safari_13,safari,macos,macos_big_sur_11,2022-01-07 02:54:32,内网,200,成都,nHrKgKdJ1Mzt,xxx.com/github,2022-01,1,xxx.com,github
45706,45706,yuanjun5870,hr,1Vk2kEa4X,W1Cstajd8x1s,firefox_78,firefox,win,win7,2022-01-07 03:00:56,内网,200,深圳,a5G25puBl9xj,hr.xxx.com/,2022-01,1,hr.xxx.com,
45901,45901,zhoutingting3694,rd,4Wj6uxLx3,H8NAVsdws95G,edge_93,edge,win,win10,2022-01-07 04:29:34,内网,200,杭州,nHrKgKdJ1Mzt,xxx.com/github,2022-01,1,xxx.com,github
43827,43827,yanglin6562,sales,eK12oQmm8,GnkVqPSy5nnl,ie_9,ie,win,win10,2022-01-07 05:17:44,内网,200,重庆,sW0whYIx8LFM,work.xxx.com/task,2022-01,1,work.xxx.com,task


In [4]:
with open(os.path.join(path_new_data, 'no_times_data.pkl'), 'rb') as f:
    df = pkl.load(f)

In [10]:
df['ip_transform'] = df_train_info['ip_transform']

In [11]:
df_tmp = pd.read_csv('/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/new_data/corpus_txt/models_device_num_transform_embedding.txt', skiprows=1, header=None, sep=' ' )
df_tmp.columns = ['corpus_{}'.format(i) for i in range(6)]
df_tmp

,corpus_0,corpus_1,corpus_2,corpus_3,corpus_4,corpus_5
0,u9diCFdYZ,1.191250,0.884346,0.960100,-0.587945,-0.437067
1,w2CfuqTz3,-0.310487,0.538982,1.129704,-1.143607,0.040968
2,YBCE8ld50,0.031544,1.011728,-0.039371,-1.519553,-0.246772
3,5KbVyNsBf,0.134305,0.212648,0.355266,-1.386184,-0.776587
4,FQND8WWo5,0.745319,0.974483,0.725129,-0.819523,-1.404120
...,...,...,...,...,...,...
3105,4XpDhY9N8,0.447430,0.264440,0.598619,-1.016775,-0.057525
3106,b8PyvUL9p,0.446785,0.555997,0.494677,-0.910030,-0.297620
3107,6CzA6Vd7a,0.249200,0.495751,0.629262,-0.913831,-0.121946
3108,6zzVVzNbn,0.279105,0.324233,0.421833,-1.026533,-0.260341


In [12]:
df.head()

,id,department,browser_version,browser,os_type,os_version,ip_type,http_status_code,op_city,log_system_transform,url,is_risk,url_sit,url_page,op_diff_second1,op_diff_second1_log,op_diff_second1_log_log,op_times_groups,system_op_diff_second1,system_op_diff_second1_log,system_op_diff_second1_log_log,system_op_times_groups,system_system_op_times_groups_cumsum,system_system_op_times_groups_error_code_cumsum,system_system_op_times_groups_ip_transform_cumunique,system_system_op_times_groups_user_name_cumunique,system_system_op_times_groups_device_num_transform_cumunique,system_system_op_times_groups_department_cumunique,system_system_op_times_groups_department_rd_cumsum,system_system_op_times_groups_department_rd_error_code_cumsum,system_system_op_times_groups_department_hr_cumsum,system_system_op_times_groups_department_hr_error_code_cumsum,system_system_op_times_groups_department_sales_cumsum,system_system_op_times_groups_department_sales_error_code_cumsum,system_system_op_times_groups_department_other_cumsum,system_system_op_times_groups_department_other_error_code_cumsum,system_system_op_times_groups_department_nan_cumsum,system_system_op_times_groups_department_nan_error_code_cumsum,system_system_op_times_groups_department_accounting_cumsum,system_system_op_times_groups_department_accounting_error_code_cumsum,system_system_op_times_groups_browser_version_cumunique,system_system_op_times_groups_browser_version_chrome_93_cumsum,system_system_op_times_groups_browser_version_chrome_93_error_code_cumsum,system_system_op_times_groups_browser_version_safari_13_cumsum,system_system_op_times_groups_browser_version_safari_13_error_code_cumsum,system_system_op_times_groups_browser_version_firefox_78_cumsum,system_system_op_times_groups_browser_version_firefox_78_error_code_cumsum,system_system_op_times_groups_browser_version_edge_93_cumsum,system_system_op_times_groups_browser_version_edge_93_error_code_cumsum,system_system_op_times_groups_browser_version_ie_9_cumsum,system_system_op_times_groups_browser_version_ie_9_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_77_cumsum,system_system_op_times_groups_browser_version_chrome_77_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_90_cumsum,system_system_op_times_groups_browser_version_chrome_90_error_code_cumsum,system_system_op_times_groups_browser_version_ie_11_cumsum,system_system_op_times_groups_browser_version_ie_11_error_code_cumsum,system_system_op_times_groups_browser_cumunique,system_system_op_times_groups_browser_chrome_cumsum,system_system_op_times_groups_browser_chrome_error_code_cumsum,system_system_op_times_groups_browser_safari_cumsum,system_system_op_times_groups_browser_safari_error_code_cumsum,system_system_op_times_groups_browser_firefox_cumsum,system_system_op_times_groups_browser_firefox_error_code_cumsum,system_system_op_times_groups_browser_edge_cumsum,system_system_op_times_groups_browser_edge_error_code_cumsum,system_system_op_times_groups_browser_ie_cumsum,system_system_op_times_groups_browser_ie_error_code_cumsum,system_system_op_times_groups_os_type_cumunique,system_system_op_times_groups_os_type_win_cumsum,system_system_op_times_groups_os_type_win_error_code_cumsum,system_system_op_times_groups_os_type_macos_cumsum,system_system_op_times_groups_os_type_macos_error_code_cumsum,system_system_op_times_groups_os_version_cumunique,system_system_op_times_groups_os_version_win10_cumsum,system_system_op_times_groups_os_version_win10_error_code_cumsum,system_system_op_times_groups_os_version_macos_big_sur_11_cumsum,system_system_op_times_groups_os_version_macos_big_sur_11_error_code_cumsum,system_system_op_times_groups_os_version_win7_cumsum,system_system_op_times_groups_os_version_win7_error_code_cumsum,system_system_op_times_groups_os_version_win11_cumsum,system_system_op_times_groups_os_version_win11_error_code_cumsum,system_system_op_times_groups_ip_type_cumunique,system_system_op_times_groups_ip_type_内网_cumsum,system_system

In [13]:
df = pd.merge(left=df, right=df_tmp, how='left', left_on='ip_transform', right_on='corpus_0')

In [14]:
# 删除时间及类别型变量过多的特征
remove_col = ['op_datetime', 'op_month', 'user_name', 
              'op_times_groups', 'system_op_times_groups', 'op_days', 'op_month',
              'ip_transform', 'device_num_transform', 'op_days', 'ts', 'ts1', 'ts2','ts3','corpus_0', 'ip_transform']

remove_col = [x for x in df.columns if x in remove_col]
df = df.drop(columns=remove_col)
df.head()

,id,department,browser_version,browser,os_type,os_version,ip_type,http_status_code,op_city,log_system_transform,url,is_risk,url_sit,url_page,op_diff_second1,op_diff_second1_log,op_diff_second1_log_log,system_op_diff_second1,system_op_diff_second1_log,system_op_diff_second1_log_log,system_system_op_times_groups_cumsum,system_system_op_times_groups_error_code_cumsum,system_system_op_times_groups_ip_transform_cumunique,system_system_op_times_groups_user_name_cumunique,system_system_op_times_groups_device_num_transform_cumunique,system_system_op_times_groups_department_cumunique,system_system_op_times_groups_department_rd_cumsum,system_system_op_times_groups_department_rd_error_code_cumsum,system_system_op_times_groups_department_hr_cumsum,system_system_op_times_groups_department_hr_error_code_cumsum,system_system_op_times_groups_department_sales_cumsum,system_system_op_times_groups_department_sales_error_code_cumsum,system_system_op_times_groups_department_other_cumsum,system_system_op_times_groups_department_other_error_code_cumsum,system_system_op_times_groups_department_nan_cumsum,system_system_op_times_groups_department_nan_error_code_cumsum,system_system_op_times_groups_department_accounting_cumsum,system_system_op_times_groups_department_accounting_error_code_cumsum,system_system_op_times_groups_browser_version_cumunique,system_system_op_times_groups_browser_version_chrome_93_cumsum,system_system_op_times_groups_browser_version_chrome_93_error_code_cumsum,system_system_op_times_groups_browser_version_safari_13_cumsum,system_system_op_times_groups_browser_version_safari_13_error_code_cumsum,system_system_op_times_groups_browser_version_firefox_78_cumsum,system_system_op_times_groups_browser_version_firefox_78_error_code_cumsum,system_system_op_times_groups_browser_version_edge_93_cumsum,system_system_op_times_groups_browser_version_edge_93_error_code_cumsum,system_system_op_times_groups_browser_version_ie_9_cumsum,system_system_op_times_groups_browser_version_ie_9_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_77_cumsum,system_system_op_times_groups_browser_version_chrome_77_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_90_cumsum,system_system_op_times_groups_browser_version_chrome_90_error_code_cumsum,system_system_op_times_groups_browser_version_ie_11_cumsum,system_system_op_times_groups_browser_version_ie_11_error_code_cumsum,system_system_op_times_groups_browser_cumunique,system_system_op_times_groups_browser_chrome_cumsum,system_system_op_times_groups_browser_chrome_error_code_cumsum,system_system_op_times_groups_browser_safari_cumsum,system_system_op_times_groups_browser_safari_error_code_cumsum,system_system_op_times_groups_browser_firefox_cumsum,system_system_op_times_groups_browser_firefox_error_code_cumsum,system_system_op_times_groups_browser_edge_cumsum,system_system_op_times_groups_browser_edge_error_code_cumsum,system_system_op_times_groups_browser_ie_cumsum,system_system_op_times_groups_browser_ie_error_code_cumsum,system_system_op_times_groups_os_type_cumunique,system_system_op_times_groups_os_type_win_cumsum,system_system_op_times_groups_os_type_win_error_code_cumsum,system_system_op_times_groups_os_type_macos_cumsum,system_system_op_times_groups_os_type_macos_error_code_cumsum,system_system_op_times_groups_os_version_cumunique,system_system_op_times_groups_os_version_win10_cumsum,system_system_op_times_groups_os_version_win10_error_code_cumsum,system_system_op_times_groups_os_version_macos_big_sur_11_cumsum,system_system_op_times_groups_os_version_macos_big_sur_11_error_code_cumsum,system_system_op_times_groups_os_version_win7_cumsum,system_system_op_times_groups_os_version_win7_error_code_cumsum,system_system_op_times_groups_os_version_win11_cumsum,system_system_op_times_groups_os_version_win11_error_code_cumsum,system_system_op_times_groups_ip_type_cumunique,system_system_op_times_groups_ip_type_内网_cumsum,system_system_op_times_groups_ip_type_内网_error_code_

In [15]:
df_row_train = df[df[y_label].notna()].reset_index(drop=True)
df_row_val = df[df[y_label].isna()].reset_index(drop=True)

df_train, df_test, convert_cols = sp.transform_data_detail(df_row_train, df_row_val, y_label, excel_path=path_output_report)
df_train.head()

sheet05.可能为数值类型的object类型数据统计在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察_20221014.xlsx中已经存在，我们将对原文件进行覆盖
sheet06.数据预处理在/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification/results/01_原始数据探察_20221014.xlsx中已经存在，我们将对原文件进行覆盖


,id,department,browser_version,browser,os_version,op_city,log_system_transform,url,is_risk,url_sit,url_page,op_diff_second1,op_diff_second1_log,op_diff_second1_log_log,system_op_diff_second1,system_op_diff_second1_log,system_op_diff_second1_log_log,system_system_op_times_groups_cumsum,system_system_op_times_groups_error_code_cumsum,system_system_op_times_groups_department_rd_cumsum,system_system_op_times_groups_department_rd_error_code_cumsum,system_system_op_times_groups_department_sales_cumsum,system_system_op_times_groups_department_sales_error_code_cumsum,system_system_op_times_groups_browser_version_edge_93_cumsum,system_system_op_times_groups_browser_version_edge_93_error_code_cumsum,system_system_op_times_groups_browser_version_chrome_90_cumsum,system_system_op_times_groups_browser_version_chrome_90_error_code_cumsum,system_system_op_times_groups_browser_chrome_cumsum,system_system_op_times_groups_browser_chrome_error_code_cumsum,system_system_op_times_groups_browser_edge_cumsum,system_system_op_times_groups_browser_edge_error_code_cumsum,system_system_op_times_groups_os_type_win_cumsum,system_system_op_times_groups_os_type_win_error_code_cumsum,system_system_op_times_groups_os_version_win10_cumsum,system_system_op_times_groups_os_version_win10_error_code_cumsum,system_system_op_times_groups_os_version_win7_cumsum,system_system_op_times_groups_os_version_win7_error_code_cumsum,system_system_op_times_groups_ip_type_内网_cumsum,system_system_op_times_groups_ip_type_内网_error_code_cumsum,system_system_op_times_groups_http_status_code_200_cumsum,system_system_op_times_groups_op_city_深圳_cumsum,system_system_op_times_groups_op_city_深圳_error_code_cumsum,system_system_op_times_groups_op_city_成都_cumsum,system_system_op_times_groups_op_city_成都_error_code_cumsum,system_system_op_times_groups_op_city_杭州_cumsum,system_system_op_times_groups_op_city_杭州_error_code_cumsum,system_system_op_times_groups_op_city_北京_cumsum,system_system_op_times_groups_op_city_北京_error_code_cumsum,system_system_op_times_groups_log_system_transform_nHrKgKdJ1Mzt_cumsum,system_system_op_times_groups_log_system_transform_nHrKgKdJ1Mzt_error_code_cumsum,system_system_op_times_groups_log_system_transform_dwS3cdn15GK4_cumsum,system_system_op_times_groups_log_system_transform_dwS3cdn15GK4_error_code_cumsum,system_system_op_times_groups_url_xxx.com/github_cumsum,system_system_op_times_groups_url_xxx.com/github_error_code_cumsum,system_system_op_times_groups_url_wpsdoc.xxx.com/download_cumsum,system_system_op_times_groups_url_wpsdoc.xxx.com/download_error_code_cumsum,system_system_op_times_groups_url_page_github_cumsum,system_system_op_times_groups_url_page_github_error_code_cumsum,system_system_op_times_groups_url_page_download_cumsum,system_system_op_times_groups_url_page_download_error_code_cumsum,system_system_op_times_groups_url_sit_xxx.com_cumsum,system_system_op_times_groups_url_sit_xxx.com_error_code_cumsum,system_system_op_times_groups_url_sit_wpsdoc.xxx.com_cumsum,system_system_op_times_groups_url_sit_wpsdoc.xxx.com_error_code_cumsum,system_op_days_cumsum,system_op_days_error_code_cumsum,system_op_days_department_rd_cumsum,system_op_days_department_rd_error_code_cumsum,system_op_days_department_sales_cumsum,system_op_days_department_sales_error_code_cumsum,system_op_days_browser_version_edge_93_cumsum,system_op_days_browser_version_edge_93_error_code_cumsum,system_op_days_browser_version_chrome_90_cumsum,system_op_days_browser_version_chrome_90_error_code_cumsum,system_op_days_browser_chrome_cumsum,system_op_days_browser_chrome_error_code_cumsum,system_op_days_browser_edge_cumsum,system_op_days_browser_edge_error_code_cumsum,system_op_days_os_type_win_cumsum,system_op_days_os_type_win_error_code_cumsum,system_op_days_os_version_win10_cumsum,system_op_days_os_version_win10_error_code_cumsum,system_op_days_os_version_win7_cumsum,system_op_days_os_version_win7_error_code_cumsum,system_op_days_ip_type_内网_cumsum,system_op_days_ip_type_内网_error_code_cumsum,system_op

In [16]:
df_tmp.columns

Index(['corpus_0', 'corpus_1', 'corpus_2', 'corpus_3', 'corpus_4', 'corpus_5'], dtype='object')

In [17]:
feats = ['day_cos', 'day_sin', 'dayofweek', 'dayofweek_sin', 'hour_cos',
       'op_days_ip_transform_cumsum',
       'op_days_ip_transform_os_type_win_error_code_cumsum',
       'op_days_ip_transform_url_sit_xxx.com_cumsum',
       'op_days_user_name_os_version_win10_error_code_cumsum',
       'op_diff_second1',
       'op_month_ip_transform_http_status_code_200_cumsum',
       'op_month_ip_transform_url_wpsdoc.xxx.com/download_cumsum',
       'op_times_groups_device_num_transform_http_status_code_200_cumsum',
       'op_times_groups_user_name_browser_chrome_error_code_cumsum',
       'op_times_groups_user_name_browser_version_edge_93_cumsum',
       'op_times_groups_user_name_error_code_cumsum',
       'op_times_groups_user_name_http_status_code_200_cumsum',
       'op_times_groups_user_name_os_type_win_error_code_cumsum',
       'system_op_days_http_status_code_200_cumsum',
       'system_op_days_op_city_北京_error_code_cumsum',
       'system_op_days_op_city_成都_error_code_cumsum',
       'system_op_days_op_city_深圳_error_code_cumsum',
       'system_op_days_os_version_win7_error_code_cumsum',
       'system_op_diff_second1_log_log',
       'system_system_op_times_groups_browser_version_edge_93_error_code_cumsum',
       'system_system_op_times_groups_http_status_code_200_cumsum',
       'system_system_op_times_groups_op_city_北京_error_code_cumsum',
       'system_system_op_times_groups_op_city_杭州_error_code_cumsum',
       'system_system_op_times_groups_op_city_深圳_error_code_cumsum',
       'system_system_op_times_groups_os_version_win7_error_code_cumsum',
        'corpus_1', 'corpus_2', 'corpus_3', 'corpus_4', 'corpus_5'
        ]
        

In [18]:

import time
from sklearn.metrics import roc_auc_score as auc
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold

In [19]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

In [24]:
feats = feats_importance.sort_values('importance', ascending=False)[:20]['name'].values

In [25]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []
for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    # kf = KFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label],
                           # categorical_feature=categorical_feats
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label],
                          #categorical_feature=categorical_feats
                         )
        model = lgb.train(params, train, valid_sets=[val], 
                          num_boost_round=20000, early_stopping_rounds=100)

        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

df_train['oof'] = oof
display(np.mean(score), np.std(score))

score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values('id').reset_index(drop=True)

sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_jupyter,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
[1]	valid_0's auc: 0.93233
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.934502
[3]	valid_0's auc: 0.934643
[4]	valid_0's auc: 0.934117
[5]	valid_0's auc: 0.938339
[6]	valid_0's auc: 0.938308
[7]	valid_0's auc: 0.939965
[8]	valid_0's auc: 0.940141
[9]	valid_0's auc: 0.940483
[10]	valid_0's auc: 0.940481
[11]	valid_0's auc: 0.940669
[12]	valid_0's auc: 0.940756
[13]	valid_0's auc: 0.941294
[14]	valid_0's auc: 0.941344
[15]	valid_0's auc: 0.941282
[16]	valid_0's auc: 0.941326
[17]	valid_0's auc: 0.941352
[18]	valid_0's auc: 0.941358
[19]	valid_0's auc: 0.941391
[20]	valid_0's auc: 0.941376
[21]	valid_0's auc: 0.941285
[22]	valid_0's auc: 0.941222
[23]	valid_0's auc: 0.941428
[24]	valid_0's auc: 0.938551
[25]	valid_0's auc: 0.938608
[26]	valid_0's auc: 0.938983
[27]	valid_0's auc: 0.94021
[28]	valid_0's auc: 0.940231
[29]	valid_0's auc: 0.940249
[30]	valid_0's auc: 0.940222
[31]	valid_0's auc: 0.939035
[32]	valid_0's auc: 0.938935
[33]	va

[174]	valid_0's auc: 0.940621
[175]	valid_0's auc: 0.940651
[176]	valid_0's auc: 0.940526
[177]	valid_0's auc: 0.940542
[178]	valid_0's auc: 0.940312
[179]	valid_0's auc: 0.940458
[180]	valid_0's auc: 0.940477
[181]	valid_0's auc: 0.940341
[182]	valid_0's auc: 0.940343
[183]	valid_0's auc: 0.940281
[184]	valid_0's auc: 0.940258
[185]	valid_0's auc: 0.940341
[186]	valid_0's auc: 0.940562
[187]	valid_0's auc: 0.94042
[188]	valid_0's auc: 0.940655
[189]	valid_0's auc: 0.940648
[190]	valid_0's auc: 0.940649
[191]	valid_0's auc: 0.940667
[192]	valid_0's auc: 0.940681
Early stopping, best iteration is:
[92]	valid_0's auc: 0.9413
----------- 2
[1]	valid_0's auc: 0.931948
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.93333
[3]	valid_0's auc: 0.931732
[4]	valid_0's auc: 0.932953
[5]	valid_0's auc: 0.9351
[6]	valid_0's auc: 0.934275
[7]	valid_0's auc: 0.934789
[8]	valid_0's auc: 0.934879
[9]	valid_0's auc: 0.937877
[10]	valid_0's auc: 0.938297
[11]	valid_0's 

[126]	valid_0's auc: 0.94227
[127]	valid_0's auc: 0.942197
[128]	valid_0's auc: 0.942109
[129]	valid_0's auc: 0.942002
[130]	valid_0's auc: 0.942218
[131]	valid_0's auc: 0.942269
[132]	valid_0's auc: 0.942161
[133]	valid_0's auc: 0.942212
[134]	valid_0's auc: 0.942602
[135]	valid_0's auc: 0.942417
[136]	valid_0's auc: 0.942644
[137]	valid_0's auc: 0.942539
[138]	valid_0's auc: 0.942301
[139]	valid_0's auc: 0.942143
[140]	valid_0's auc: 0.941974
[141]	valid_0's auc: 0.941916
[142]	valid_0's auc: 0.941895
[143]	valid_0's auc: 0.941909
[144]	valid_0's auc: 0.941786
[145]	valid_0's auc: 0.941701
[146]	valid_0's auc: 0.941467
[147]	valid_0's auc: 0.941515
[148]	valid_0's auc: 0.941566
[149]	valid_0's auc: 0.941464
[150]	valid_0's auc: 0.941349
[151]	valid_0's auc: 0.941271
[152]	valid_0's auc: 0.941363
[153]	valid_0's auc: 0.941436
[154]	valid_0's auc: 0.941377
[155]	valid_0's auc: 0.941271
[156]	valid_0's auc: 0.941241
[157]	valid_0's auc: 0.941176
[158]	valid_0's auc: 0.941117
[159]	valid

,name,importance
0,system_system_op_times_groups_http_status_code...,75945.970980
4,system_op_days_http_status_code_200_cumsum,33424.719153
1,op_month_ip_transform_http_status_code_200_cumsum,30065.491083
2,op_times_groups_device_num_transform_http_stat...,15917.368443
5,hour_cos,11125.789832
3,op_diff_second1,10237.530358
6,system_op_diff_second1_log_log,7059.821205
7,op_times_groups_user_name_http_status_code_200...,4571.573572
8,op_month_ip_transform_url_wpsdoc.xxx.com/downl...,2756.070752
9,corpus_2,1940.398058


0.9416226699396841

0.0020374066207475548

In [28]:
feats_importance.sort_values('importance', ascending=False).shape

(35, 2)

In [29]:
tmp = feats_importance.sort_values('importance', ascending=False)['name'].values
tmp.sort()
for i in tmp:
    print(i)

corpus_1
corpus_2
corpus_3
corpus_4
corpus_5
day_cos
day_sin
dayofweek
dayofweek_sin
hour_cos
op_days_ip_transform_cumsum
op_days_ip_transform_os_type_win_error_code_cumsum
op_days_ip_transform_url_sit_xxx.com_cumsum
op_days_user_name_os_version_win10_error_code_cumsum
op_diff_second1
op_month_ip_transform_http_status_code_200_cumsum
op_month_ip_transform_url_wpsdoc.xxx.com/download_cumsum
op_times_groups_device_num_transform_http_status_code_200_cumsum
op_times_groups_user_name_browser_chrome_error_code_cumsum
op_times_groups_user_name_browser_version_edge_93_cumsum
op_times_groups_user_name_error_code_cumsum
op_times_groups_user_name_http_status_code_200_cumsum
op_times_groups_user_name_os_type_win_error_code_cumsum
system_op_days_http_status_code_200_cumsum
system_op_days_op_city_北京_error_code_cumsum
system_op_days_op_city_成都_error_code_cumsum
system_op_days_op_city_深圳_error_code_cumsum
system_op_days_os_version_win7_error_code_cumsum
system_op_diff_second1_log_log
system_system_op_